<a href="https://colab.research.google.com/github/itssofiarce/AA1-TUIA-Petetta-Arce/blob/main/TP_integrador_AA1_Petetta_Arce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#!pip3 install shap

In [ ]:
#!pip install optuna

In [ ]:
#!pip install tensorflow

In [ ]:
#!pip install pandas

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from math import sqrt

# De scikit-learn...

# Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PowerTransformer

# Estandarización y modelado
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn import metrics

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import make_column_selector
# Division de los datos
from sklearn.model_selection import train_test_split, RandomizedSearchCV

# Normalización
from sklearn.preprocessing import MinMaxScaler

# Regresiones
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso, ElasticNet, RidgeCV, ElasticNetCV, LassoCV, LogisticRegression

# ML metricas
from sklearn.metrics import mean_squared_error, r2_score, classification_report, roc_curve, auc, confusion_matrix, accuracy_score, ConfusionMatrixDisplay, mean_absolute_percentage_error, mean_absolute_error

# Validación Cruzada
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold, LeaveOneOut

# Explicabilidad
import shap

# Redes Neuronales
from setuptools import setup
from distutils.core import setup
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import Precision
from tensorflow.keras.utils import to_categorical
from keras.optimizers import Adam

# Optimización de HP
import optuna
from optuna.trial import Trial


In [ ]:
path= 'weatherAUS.csv'
#path = '/content/drive/MyDrive/weatherAUS.csv'
df = pd.read_csv(path, usecols=range(1,25))
df.head()

In [ ]:
### Columnas
df.columns

In [ ]:
df.describe()

In [ ]:
### Cantidad de valores faltantes.
df.isna().sum()

In [ ]:
df.info()

In [ ]:
df['Location'].unique()

In [ ]:
costa_este = ["Adelaide", "Canberra", "Cobar", "Dartmoor", "Melbourne", "MelbourneAirport", "MountGambier", "Sydney", "SydneyAirport"]

df.loc[df['Location'].isin(costa_este), 'Location'] = 'costa_este'

df = df[df['Location'] == 'costa_este']

df = df.reset_index(drop=True)

In [ ]:
df.head()

In [ ]:
### Busco duplicados
if not df.duplicated().any():
  print('No hay registros duplicados')

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.describe()

# Analisis Exploratorio

## Correlación

In [ ]:
num_col = df.select_dtypes(include=['float64'])
corr = num_col.corr().round(1)
plt.figure(figsize=(15, 15))
sns.heatmap(corr, annot=True, cmap="rocket", center=0)
plt.show()

In [ ]:
### scatter_plots entre variables que tienen una alta correlación
### Humedad
num_col['RainTomorrow'] = df['RainTomorrow']
plt.figure(dpi=120)
sns.scatterplot(num_col, x='Humidity9am', y='Humidity3pm', hue='RainTomorrow', alpha=0.3)


Segun el gráfico, se observa si ambos valores de humedad son altos, entonces hay grandes chances de lluvia

In [ ]:
### Presión
plt.figure(dpi=120)
sns.scatterplot(num_col, x='Pressure9am', y='Pressure3pm', hue='RainTomorrow', alpha=0.3)

Segun el gráfico, si la presión de las 9am es baja al igual que la de las 3pm entonces hay grandes chances de lluvia

In [ ]:
### Temperatura
plt.figure(dpi=120)
sns.scatterplot(num_col, x='MaxTemp', y='MinTemp', hue='RainTomorrow', alpha=0.3)

In [ ]:
# Relación entre la Humedad a las 3pm y la cantidad de lluvia de mañana
f ,ax = plt.subplots()
plt.figure(figsize=(3, 2))
ax.scatter(num_col['Humidity3pm'], num_col['RainfallTomorrow'], s= 10)
ax.set_xlabel('Humidity3pm')
ax.set_ylabel('RainfallTomorrow')
plt.show()

Segun el gráfico, se observa una tendencia de que a mayor humedad de las 3pm mayor cantidad de lluvia

In [ ]:
# Distribución de RainfallTomorrow
sns.kdeplot(num_col['RainfallTomorrow'], bw_method=0.5)

Segun el gráfico, se observan gran cantidad de outliers

In [ ]:
#  QQ-plot
sm.qqplot((num_col['RainfallTomorrow']))
plt.title("Gráfico de Probabilidad Normal (QQ-plot)")
plt.show()

La distribución de la variable a predecir no presenta normalidad

## Pipeline

* Descartar Unnamed y Date porque son features que no vamos a utilizar: **ColDropper** (Ademas droppear RainfallTomorrow para prevenir la fuga de datos)
* Descartar todas las location que no son necesarias: **LocDropper**
* Dropear nulos y Label Encoding para las variables yes/no: **BoolYNDropperEncoder**
* Imputar valores nulos en variables categoricas con la moda: **CatFiller**
* Imputar valores nulos en variables numericas con la media:  **NumFiller**
* One Hot Encoding para las location: **LocEncoder**
* Encoding en sin y cos para WinDir: **CoordRecat**
* Estandarizar valores: **Standarizer**


In [ ]:
class ColDropper(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
        return self

  def transform(self, X):
    return X.drop(['Unnamed: 0', 'Date', 'RainTomorrow'], axis=1)

In [ ]:
class LocDropper(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
        return self

  def transform(self, X):
    aus_loc = [' Adelaide', 'Canberra', 'Cobar', 'Dartmoor', 'Melbourne', 'MelbourneAirport', 'MountGambier', 'Sydney', 'SydneyAirport' ]
    return X[X['Location'].isin(aus_loc)]

In [ ]:
class CatFiller(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
      X['WindGustDir'] = X.groupby('Location')['WindGustDir'].transform(lambda x: x.fillna(x.mode()[0]))
      X['WindDir9am'] = X.groupby('Location')['WindDir9am'].transform(lambda x: x.fillna(x.mode()[0]))
      X['WindDir3pm'] = X.groupby('Location')['WindDir3pm'].transform(lambda x: x.fillna(x.mode()[0]))

      return X

In [ ]:
class NumFiller(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
      remanining_vnul_columns = X.columns[X.isna().any()].tolist()
      for col in remanining_vnul_columns:
        X[col] =  X[col].fillna(X[col].mean())

      return X

In [ ]:
# https://www.mdpi.com/2078-2489/13/4/163 Como las variables de la dirección de los vientos pueden tener hasta 16 direcciones diferentes, para convertirlos a variables numéricas, se tiene encuenta una distribución circular. Por eso, cada una de las variables se dividió en dos: Una con el seno y otra con el coseno del angulo
class CoordRecat(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        coord = {
            'N': 0, 'NNE': 22.5, 'NE': 45, 'ENE': 67.5,
            'E': 90, 'ESE': 112.5, 'SE': 135, 'SSE': 157.5,
            'S': 180, 'SSW': 202.5, 'SW': 225, 'WSW': 247.5,
            'W': 270, 'WNW': 292.5, 'NW': 315, 'NNW': 337.5,
        }

        # Aplicar la recategorización
        for col in ['WindGustDir', 'WindDir9am', 'WindDir3pm']:
            X[col] = X[col].map(coord)
            X[f'{col}_rad'] = np.deg2rad(X[col])
            X[f'{col}_sin'] = np.sin(X[f'{col}_rad']).round(5)
            X[f'{col}_cos'] = np.cos(X[f'{col}_rad']).round(5)

        # Eliminar columnas originales y columnas radianes
        columns_to_drop = [f'{col}_rad' for col in ['WindGustDir', 'WindDir9am', 'WindDir3pm']] + ['WindGustDir', 'WindDir9am', 'WindDir3pm']
        X = X.drop(columns=columns_to_drop, axis=1)

        return X

In [ ]:
class LocEncoder(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
      dummies = pd.get_dummies(X['Location'], dtype=int)
      X = pd.concat([X, dummies], axis=1)
      X.drop('Location', axis=1, inplace=True)

      return X

In [ ]:
class ResetIndex(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.reset_index(drop=True)

In [ ]:
class Standarizer(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
      return self
  def transform(self, X):
    # Exclusión de variables booleanas y RainfallTmorrow porque no serán estandarizaradas
    exc_c = ['RainToday', 'RainTomorrow', 'RainfallTomorrow']

    # Estandarización
    df_sub = X[[col for col in X.columns if col not in exc_c]]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df_sub)

    X_scaled = pd.DataFrame(X_scaled, columns=df_sub.columns)
    for col in exc_c:
      X_scaled[f'{col}'] = X[col]

    # Nuevo DataFrame estandarizado con los nombres de las columnas originales
    return X_scaled

In [ ]:
class OutliersTreater(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
      return self
  def transform(self, X):
    cols_with_ouliers=['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm',
       'Temp9am', 'Temp3pm']

    for col in cols_with_ouliers:
      IQR=X[col].quantile(0.75)-X[col].quantile(0.25)
      lower_bridge=X[col].quantile(0.25)-(IQR*1.5)
      upper_bridge=X[col].quantile(0.75)+(IQR*1.5)

      X.loc[X[col]>=round(upper_bridge,2),col]=round(upper_bridge,2)
      X.loc[X[col]<=round(lower_bridge,2),col]=round(lower_bridge,2)

    return X

In [ ]:
class ColDropper(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
        return self

  def transform(self, X):
    return X.drop(['Date'], axis=1)

In [ ]:
class LocDropper(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
        return self

  def transform(self, X):
    costa_este = [' Adelaide', 'Canberra', 'Cobar', 'Dartmoor', 'Melbourne', 'MelbourneAirport', 'MountGambier', 'Sydney', 'SydneyAirport' ]
    X.loc[X['Location'].isin(costa_este), 'Location'] = 'costa_este'
    return X[X['Location'] == 'costa_este']

In [ ]:
class CatFiller(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
      X['WindGustDir'] = X.groupby('Location')['WindGustDir'].transform(lambda x: x.fillna(x.mode()[0]))
      X['WindDir9am'] = X.groupby('Location')['WindDir9am'].transform(lambda x: x.fillna(x.mode()[0]))
      X['WindDir3pm'] = X.groupby('Location')['WindDir3pm'].transform(lambda x: x.fillna(x.mode()[0]))

      return X

In [ ]:
class NumFiller(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
      remanining_vnul_columns = X.columns[X.isna().any()].tolist()
      for col in remanining_vnul_columns:
        X[col] =  X[col].fillna(X[col].mean())

      return X

In [ ]:
# https://www.mdpi.com/2078-2489/13/4/163 Como las variables de la dirección de los vientos pueden tener hasta 16 direcciones diferentes, para convertirlos a variables numéricas, se tiene encuenta una distribución circular. Por eso, cada una de las variables se dividió en dos: Una con el seno y otra con el coseno del angulo
class CoordRecat(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        coord = {
            'N': 0, 'NNE': 22.5, 'NE': 45, 'ENE': 67.5,
            'E': 90, 'ESE': 112.5, 'SE': 135, 'SSE': 157.5,
            'S': 180, 'SSW': 202.5, 'SW': 225, 'WSW': 247.5,
            'W': 270, 'WNW': 292.5, 'NW': 315, 'NNW': 337.5,
        }

        # Aplicar la recategorización
        for col in ['WindGustDir', 'WindDir9am', 'WindDir3pm']:
            X[col] = X[col].map(coord)
            X[f'{col}_rad'] = np.deg2rad(X[col])
            X[f'{col}_sin'] = np.sin(X[f'{col}_rad']).round(5)
            X[f'{col}_cos'] = np.cos(X[f'{col}_rad']).round(5)

        # Eliminar columnas originales y columnas radianes
        columns_to_drop = [f'{col}_rad' for col in ['WindGustDir', 'WindDir9am', 'WindDir3pm']] + ['WindGustDir', 'WindDir9am', 'WindDir3pm']
        X = X.drop(columns=columns_to_drop, axis=1)

        return X

In [ ]:
class LocEncoder(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
      dummies = pd.get_dummies(X['Location'], dtype=int)
      X = pd.concat([X, dummies], axis=1)
      X.drop('Location', axis=1, inplace=True)

      return X

In [ ]:
class ResetIndex(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.reset_index(drop=True)

In [ ]:
class BoolYNDropperEncoder(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
      X.dropna(subset=['RainToday'], inplace=True)
      X['RainTomorrow'] = X['RainTomorrow'].map({'No': 0, 'Yes': 1}).astype(float)
      X['RainToday'] = X['RainToday'].map({'No': 0, 'Yes': 1}).astype(float)

      return X

In [ ]:
class Standarizer(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
      return self
  def transform(self, X):
    # Exclusión de variables booleanas y RainfallTmorrow porque no serán estandarizaradas
    exc_c = ['RainToday', 'RainTomorrow']

    # Estandarización
    df_sub = X[[col for col in X.columns if col not in exc_c]]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df_sub)

    X_scaled = pd.DataFrame(X_scaled, columns=df_sub.columns)
    for col in exc_c:
      X_scaled[f'{col}'] = X[col]

    # Nuevo DataFrame estandarizado con los nombres de las columnas originales
    return X_scaled

In [ ]:
class OutliersTreater(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
      return self
  def transform(self, X):
    cols_with_ouliers=['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm',
       'Temp9am', 'Temp3pm']

    for col in cols_with_ouliers:
      IQR=X[col].quantile(0.75)-X[col].quantile(0.25)
      lower_bridge=X[col].quantile(0.25)-(IQR*1.5)
      upper_bridge=X[col].quantile(0.75)+(IQR*1.5)

      X.loc[X[col]>=round(upper_bridge,2),col]=round(upper_bridge,2)
      X.loc[X[col]<=round(lower_bridge,2),col]=round(lower_bridge,2)

    return X


In [ ]:
class RLValDropper(BaseEstimator, TransformerMixin):
  def fit (self, X, y=None):
    return self
  def transform(self, X):
    X.dropna(subset=['RainTomorrow'], inplace=True)
    return X


In [ ]:
preprocessor = Pipeline([
     ('drop_null_val_rl', RLValDropper()),
     ('drop_not_needed_features', ColDropper()),
     ('drop_nor_needed_locations',LocDropper()),
     ('yes_no_dropper_encoder', BoolYNDropperEncoder()),
     ('fill_null_cat', CatFiller()),
     ('fill_num_cat', NumFiller()),
     ('encode_loc', LocEncoder()),
     ('encode_wind_dir', CoordRecat()),
     ('reset_index',ResetIndex()),
     ('treat_outliers',OutliersTreater()),
     ('standariza_values', Standarizer())
])


# Split Train-Test

In [ ]:
# Dropeo valores nulos de 'RainfallTomorrow' de mi dataframe original
df.dropna(subset=['RainToday', 'RainTomorrow'], inplace=True)

# Separación de variables explicativas y variables objetivo
X = df.drop(['RainfallTomorrow'], axis=1).copy()
y = df[['RainfallTomorrow']].copy()

# Spliteo mi dataset en train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# Creo un Dataframe de TRAIN
df_train = pd.DataFrame(X_train, columns=X.columns)
df_train['RainfallTomorrow'] = y['RainfallTomorrow']

# Creo un Dataframe de TEST
df_test = pd.DataFrame(X_test, columns=X.columns)

df_test['RainfallTomorrow'] = y['RainfallTomorrow']

# Preproceso mi df de test y mi df de train
df_train = preprocessor.fit_transform(df_train)
df_test = preprocessor.fit_transform(df_test)

In [ ]:
num_cols = df_train.select_dtypes(include=['float64']).columns.tolist()
for col in num_cols:
  plt.figure(figsize=(6, 2))
  df_train[col].plot(kind='box', vert=False)
  plt.tight_layout()
  plt.show()

# Regresiones


Modelo que predice la cantidad de lluvia para el día siguiente. "RainfallTomorrow" --> varibale continua a predecir.

In [ ]:
## Con features correlacionadas >= 0.1
X_train = df_train[['RainToday', 'Humidity3pm', 'Rainfall', 'WindGustSpeed', 'Cloud9am', 'Cloud3pm', 'MinTemp', 'WindSpeed9am', 'WindSpeed3pm']]
y_train = df_train['RainfallTomorrow']

X_test = df_test[['RainToday', 'Humidity3pm', 'Rainfall', 'WindGustSpeed', 'Cloud9am', 'Cloud3pm', 'MinTemp', 'WindSpeed9am', 'WindSpeed3pm']]
y_test = df_test['RainfallTomorrow']

In [ ]:
# Modelo de regresión lineal
linear_model = LinearRegression()
# Entrenamiento del modelo
linear_model.fit(X_train, y_train)
# Predicciones en el conjunto de prueba
y_pred = linear_model.predict(X_test)

# Regresión Lineal

In [ ]:
# Métricas del modelo
mse_selfeat = mean_squared_error(y_test, y_pred)
r2_selfeat = r2_score(y_test, y_pred)
score_selfeat = linear_model.score(X_test, y_test)
score_selfeat_train = linear_model.score(X_train, y_train)
print(f"Puntaje train: {score_selfeat_train}")
print(f"Puntaje test: {score_selfeat}")

print(f'Error cuadrático medio (MSE) con features seleccionadas: {mse_selfeat:.2f}')
print(f'Coeficiente de determinación (R^2) con features seleccionadas: {r2_selfeat:.2f}\n')

In [ ]:
fig = plt.figure(figsize=(8,4))
fig.suptitle('residuos')
plt.xlabel('y_test')
plt.ylabel('residuos')
plt.scatter(y_test,100*(y_test-y_pred)/y_test)
plt.show()

El patrón nos sugiere que la función de regresión no es lineal, hay sesgo. Simplificamos el modelo para mejorar las métricas

In [ ]:
# Características (X) y la variable objetivo (y)
X_train = df_train['Humidity3pm'].values.reshape(-1, 1)
y_train = df_train['RainfallTomorrow'].values

# Características (X) y la variable objetivo (y) para el conjunto de prueba
X_test = df_test[['Humidity3pm']].values.reshape(-1, 1)
y_test = df_test['RainfallTomorrow'].values

In [ ]:
# Modelo de regresión lineal
linear_model = LinearRegression()
# Entrenamiento del modelo
linear_model.fit(X_train, y_train)
# Predicciones en el conjunto de prueba
y_pred = linear_model.predict(X_test)

# Métricas del modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

score_selfeat = linear_model.score(X_test, y_test)
score_selfeat_train = linear_model.score(X_train, y_train)
print(f"Puntaje train: {score_selfeat_train}")
print(f"Puntaje test: {score_selfeat}")

print(f'Error cuadrático medio (MSE): {mse:.2f}')
print(f'Coeficiente de determinación (R^2): {r2:.2f}\n')

In [ ]:
plt.scatter(X_train, y_train, color='blue')
plt.plot(X_train, linear_model.predict(X_train), color='red')

plt.xlabel('Humidity3pm')
plt.ylabel('RainfallTomorrow')
plt.title('Linear Regression Model')

plt.show()

## Regresión lineal con gradiente descendiente

In [ ]:
## Con todas las features
X_train = df_train.drop(['RainfallTomorrow'], axis=1)
y_train = df_train['RainfallTomorrow']

X_test = df_test.drop(['RainfallTomorrow'], axis=1)
y_test = df_test['RainfallTomorrow']

In [ ]:
# Creación del modelo
model_sgd = SGDRegressor(max_iter=10000, random_state=42)

# Listas para almacenar las pérdidas
training_losses = []
validation_losses = []

# Ciclo de entrenamiento
for epoch in range(1000):  # Ajusta según el número real de épocas
    # train
    model_sgd.partial_fit(X_train, y_train)
    y_train_pred = model_sgd.predict(X_train)
    training_loss = mean_squared_error(y_train, y_train_pred)
    training_losses.append(training_loss)

    # Validación
    y_val_pred = model_sgd.predict(X_test)
    validation_loss = mean_squared_error(y_test, y_val_pred)
    validation_losses.append(validation_loss)

# Graficar las pérdidas
plt.plot(training_losses, label='Training Loss')
plt.plot(validation_losses, label='Validation Loss')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.legend()
plt.show()


# Predicciones en el conjunto de prueba (Test)
y_pred_sgd = model_sgd.predict(X_test)

# Métricas del modelo
mse_sgd = mean_squared_error(y_test, y_pred_sgd)
r2_sgd = r2_score(y_test, y_pred_sgd)

# Outputs
print('Resultados con Gradiente Descendiente:')
print(f'Error cuadrático medio (MSE): {mse_sgd:.2f}')
print(f'Coeficiente de determinación (R^2): {r2_sgd:.2f}')

Hay una mejora para ambas métricas en comparación con la regresión lineal sin regularización. El R-cuadrado aumenta en unos 23 centecimas, lo que indica un mejor ajuste.

## LR con Regularización

### Lasso

In [ ]:
## Con todas las features
X_train = df_train.drop(['RainfallTomorrow'], axis=1)
y_train = df_train['RainfallTomorrow']

X_test = df_test.drop(['RainfallTomorrow'], axis=1)
y_test = df_test['RainfallTomorrow']

In [ ]:
modelo_lasso = make_pipeline(StandardScaler(),
                        LassoCV(
                           alphas=np.logspace(-10, 10, 200),
                           cv=10
                        )
         )

_ = modelo_lasso.fit(X_train, y_train)

In [ ]:
# Evolución de los parámetros en función de alpha
alphas = modelo_lasso.steps[1][1].alphas
coefs = []

for alpha in alphas:
    modelo_aux = make_pipeline(StandardScaler(), Lasso(alpha=alpha, tol=1))
    modelo_aux.fit(X_train, y_train)
    coefs.append(modelo_aux.steps[1][1].coef_.flatten())

fig, ax = plt.subplots(figsize=(15,5))
ax.plot(alphas, coefs, label=X_train.columns)
ax.set_xscale('log')
ax.set_ylim([-0.5,None])
ax.set_xlabel('alpha')
ax.set_ylabel('parámetros')
ax.legend()
ax.set_title('Coeficientes del modelo en función de la regularización')
plt.show()

In [ ]:
n_predictores = []

for alpha in alphas:
    modelo_aux = make_pipeline(StandardScaler(), Lasso(alpha=alpha, tol=1))
    modelo_aux.fit(X_train, y_train)
    coef_no_cero = np.sum(modelo_aux.steps[1][1].coef_.flatten() != 0)
    n_predictores.append(coef_no_cero)

fig, ax = plt.subplots(figsize=(15, 5))
ax.plot(alphas, n_predictores)
ax.set_xscale('log')
ax.set_ylim([-0.5,None])
ax.set_xlabel('alpha')
ax.set_ylabel('nº parámetros')
ax.set_title('Features incluidas en función de la regularización')

In [ ]:
# Mejor valor alpha encontrado
mod = modelo_lasso.steps[1][1]
print(f"Mejor valor de alpha encontrado: {mod.alpha_}")

In [ ]:
# Error de test del modelo
rmse_lasso = mean_squared_error(y_test,y_pred,squared = False)
R2_lasso = metrics.r2_score(y_test, y_pred)

print(f"El error (rmse) de test es: {rmse_lasso}")
print(f"El error (R2) de test es: {R2_lasso}")

### Ridge

In [ ]:
## Con todas las features
X_train = df_train.drop(['RainfallTomorrow'], axis=1)
y_train = df_train['RainfallTomorrow']

X_test = df_test.drop(['RainfallTomorrow'], axis=1)
y_test = df_test['RainfallTomorrow']

In [ ]:
modelo_ridge = make_pipeline(StandardScaler(),
                        RidgeCV(
                           alphas          = np.logspace(-10, 10, 200),
                           fit_intercept   = True,
                           store_cv_values = True
                        )
         )

_ = modelo_ridge.fit(X_train, y_train)

In [ ]:
alphas = modelo_ridge.steps[1][1].alphas
coefs = []

for alpha in alphas:
    modelo_aux = make_pipeline(StandardScaler(), Ridge(alpha=alpha))
    modelo_aux.fit(X_train, y_train)
    coefs.append(modelo_aux.steps[1][1].coef_.flatten())

fig, ax = plt.subplots(figsize=(15, 5))
ax.plot(alphas, coefs, label=X_train.columns)
ax.set_xscale('log')
ax.set_xlabel('log(alpha)')
ax.set_ylabel('Parámetros')
ax.set_title('Parámetros del modelo en función de la regularización');
plt.axis('tight')
plt.legend()
plt.show()

In [ ]:
print(f"Mejor valor de alpha encontrado: {modelo_ridge.steps[1][1].alpha_}")

In [ ]:
df_coeficientes = pd.DataFrame({'predictor': X_train.columns,'coef': mod.coef_.flatten()})

fig, ax = plt.subplots(figsize=(15,5))
ax.stem(df_coeficientes.predictor, df_coeficientes.coef, markerfmt=' ')
plt.xticks(rotation=90, ha='right', size=10)
ax.set_xlabel('feature')
ax.set_ylabel('weights')
ax.set_title('Parámetros del modelo');

In [ ]:
# Predicciones test
y_pred = modelo_ridge.predict(X_test)

In [ ]:
# Error de test del modelo
rmse_ridge = mean_squared_error(y_test,
                y_pred,
                squared = False
             )
R2_ridge = metrics.r2_score(y_test, y_pred)

print(f"El error (rmse) de test es: {rmse_ridge}")
print(f"El error (R) de test es: {R2_ridge}")

### Elastic Net

In [ ]:
## Con todas las features
X_train = df_train.drop(['RainfallTomorrow'], axis=1)
y_train = df_train['RainfallTomorrow']

X_test = df_test.drop(['RainfallTomorrow'], axis=1)
y_test = df_test['RainfallTomorrow']

In [ ]:
model_elastic = ElasticNet(alpha=0.05, l1_ratio=0.5)
alpha_EN=model_elastic.get_params()['alpha']
model_elastic.fit(X_train, y_train)

In [ ]:
y_pred_elastic = model_elastic.predict(X_test)

In [ ]:
mse_elastic = mean_squared_error(y_test, y_pred_elastic)
r2_elastic = r2_score(y_test, y_pred_elastic)

In [ ]:
print("----------")
print(f'Resultados con Regularización (Elastic Net) con alpha = {alpha_EN}')
print(f'Error cuadrático medio (MSE): {mse_elastic:.2f}')
print(f'Coeficiente de determinación (R^2): {r2_elastic:.2f}')

In [ ]:
print("Regresión Lineal:")
print("----------")
print(f'Error cuadrático medio (MSE): {mse:.2f}')
print(f'Coeficiente de determinación (R^2): {r2:.2f}\n')
print("Regresión Lineal con gradiente descendiente:")
print("----------")
print('Resultados con Gradiente Descendiente:')
print(f'Error cuadrático medio (MSE): {mse_sgd:.2f}')
print(f'Coeficiente de determinación (R^2): {r2_sgd:.2f}\n')
print("\nCon regularizaciones\n")
print('Lasso (L1):')
print("----------")
print(f"El error (rmse) de test es: {rmse_lasso}")
print(f"El error (R^2) de test es: {R2_lasso}\n")
print('Ridge (L2):')
print("----------")
print(f"El error (rmse) de test es: {rmse_ridge}")
print(f"El error (R^2) de test es: {R2_ridge}\n")
print(f'Elastic Net: alpha = {alpha_EN}')
print("----------")
print(f'Error cuadrático medio (MSE): {mse_elastic:.2f}')
print(f'Coeficiente de determinación (R^2): {r2_elastic:.2f}\n')

El modelo con mejores métricas para el problema de regresión lineal es: Regresión Lineal con gradiente descendiente ya que tiene mejor R-cuadrado

# Regresion logistica sin balancear

Decidimos hacer un analísis de los modelos de regresión con los datos balanceados y no balanceados y observar si hay alguna mejora al balancear las clases

In [ ]:
# Separación de variables explicativas y variables objetivo
X = df.drop(['RainTomorrow', 'RainfallTomorrow'], axis=1).copy()
y = df[['RainTomorrow']].copy()

# Spliteo mi dataset en train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# Creo un Dataframe de TRAIN
df_train = pd.DataFrame(X_train, columns=X.columns)
df_train['RainTomorrow'] = y['RainTomorrow']

In [ ]:
# Creo un Dataframe de TEST
df_test = pd.DataFrame(X_test, columns=X.columns)
df_test['RainTomorrow'] = y['RainTomorrow']

In [ ]:
# Preproceso mi df de test y mi df de train
df_train = preprocessor.fit_transform(df_train)
df_test = preprocessor.fit_transform(df_test)

In [ ]:
df_train.head()

In [ ]:
train_clasificacion_X = df_train.iloc[:, :-1]
y_train_clasificacion = df_train.iloc[:,-1]

In [ ]:
x_test_clasificacion = df_test.iloc[:, :-1]
y_test_clasificacion = df_test.iloc[:,-1]

In [ ]:
classifier = LogisticRegression(random_state = 0)
classifier.fit(train_clasificacion_X, y_train_clasificacion)
valor_y_predecir = classifier.predict(x_test_clasificacion)

#metricas
cm = confusion_matrix(y_test_clasificacion, valor_y_predecir)
acc_score = accuracy_score(y_test_clasificacion, valor_y_predecir)

In [ ]:
probabilidades_y = classifier.predict_proba(x_test_clasificacion)[:,-1]
fpr, tpr, thresholds = roc_curve(y_test_clasificacion, probabilidades_y)
roc_auc = auc(fpr,tpr)

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(fpr, tpr, color="red", lw=2, label='CURVA ROC (AUC = %0.2f)' % roc_auc)
plt.plot([0,1],[0,1], color="blue", lw=2, linestyle="--")
plt.xlim([0.0, 1.10])
plt.ylim([0.0, 1.10])
plt.xlabel("Tasa de Falsos positivos(FPR)")
plt.ylabel("Tasa de Verdaderos positivos(TPR)")
plt.title("ROC")
plt.legend(loc="lower right")
plt.show()

Podemos concluir que para este modelo (regresión logística) entrenado con todas las features y los datos sin balancear, tiene valor cercano a uno, lo que siginifica que el TPR (ratio de verdaderos positivos) es mas alto que el FPR (ratio de falsos positivos, es decir, predice días de no lluvia como lluvia)

# Explicabilidad con dataset no balanceado

# Explicabilidad de modelo de regresion logistica sin balancear

In [ ]:
clasificacion_parametros = {
    'C': [0.001, 0.01, 0.1, 1, 10],
    'penalty': ['l2'],
    'solver': ['liblinear', 'lbfgs', 'saga'],
    'max_iter': [100, 200, 300],
    'class_weight': [None, 'balanced']
}

optimizar_clasificacion = RandomizedSearchCV(
    LogisticRegression(random_state=23), clasificacion_parametros
)

optimizar_clasificacion.fit(train_clasificacion_X, y_train_clasificacion)

# Métricas para datos NO balanceados

In [ ]:
modelo_optimizado_clasificacion = optimizar_clasificacion.best_estimator_

In [ ]:
y_pred_clasificacion_optimizado = modelo_optimizado_clasificacion.predict(x_test_clasificacion)


print(classification_report(y_test_clasificacion, y_pred_clasificacion_optimizado))


ConfusionMatrixDisplay(
    confusion_matrix(y_test_clasificacion, y_pred_clasificacion_optimizado)
).plot()

Tiene una exactitud del 83% el modelo con los parametro optmizados con GridCV

In [ ]:
explicar_clasificacion = shap.LinearExplainer(
    classifier,
    train_clasificacion_X.astype("float64").copy(),
    features_names=train_clasificacion_X.columns,
)


shap_valores_clasificacion = explicar_clasificacion(
    train_clasificacion_X.astype("float64")
)

shap.summary_plot(shap_valores_clasificacion, train_clasificacion_X)

Las dos variables mas importantes para mi modelo son la Presión y la humedad de las 3pm. A menor presión mayor es el impacto en el modelo, osea es mas probable de que lo clasifique como un día de lluvia. Sin embargo con los valores de la humedad es al reves, A mayor humedad mayor es el impacto de la variable para el modelo prediga un dia de lluvia.

In [ ]:
index = 50

classifier.predict(x_test_clasificacion)[index]

shap_valores_clasificacion = explicar_clasificacion(x_test_clasificacion)

explicacion = shap.Explanation(
    values=shap_valores_clasificacion[index],
    base_values=explicar_clasificacion.expected_value,
    feature_names=train_clasificacion_X.columns,
)


shap.plots.waterfall(explicacion)

El valor SHAP para cada variable cuantifica cuánto impacto tiene mi variable en el modelo. Por ejemplo, según el gráfico, una presion a las 3 pm de 2.47 empuja el modelo hacia predecir menos cm cúbicos de lluvia. Caso contrario para un valor de 2.5 para la variable de presión a las 9am que contribuye a que mi modelo prediga mas cantidad de lluvia. En conclusión, para este ejemplo predijo una lluvia de -1.726 cm cúbicos.

### Interpretabilidad Global

In [ ]:
explicabilidad_global_clasificacion = shap.Explanation(
    shap_valores_clasificacion,
    base_values=explicar_clasificacion.expected_value,
    feature_names=train_clasificacion_X.columns,
    data=x_test_clasificacion,
)


shap.plots.bar(explicabilidad_global_clasificacion, max_display=None)

Al calcular el la media del valor absoluto del SHAP entonces obtenemos que las variables que mas impactan en nuestro modelo ya sea a predecir un dia de lluvia como uno de no lluvia son La presión de las 3pm y la humedad de las 3pm

Las variables con mayor impacto en nuestro modelo de regresión son Presion 3pm, Presion 9am y la direccion del viento mientras que para el modelo de clasificación son Presion 3pm, HUmedad 3pm y Presion 9am

# Modelo de regresión logistica balanceado

## Balanceo

In [ ]:
# Modulos para el balanceo
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
def plot_balance(df):
  ### Balance de datos:
  rain_classes = pd.value_counts(df['RainTomorrow'], sort = True)
  rain_classes.plot(kind = 'bar', rot=0)
  plt.xticks(range(2))
  plt.title("Frecuencia")
  plt.xlabel("RainTomorrow")
  plt.ylabel("Cantidad")
  X_train_resampled=df.drop(['RainTomorrow'], axis=1)
  y_train_resampled=df['RainTomorrow']
  print(X_train_resampled.shape, y_train_resampled.shape)
  plt.show()

In [ ]:
# Helper function for plotting ROC -- > https://towardsdatascience.com/class-imbalance-strategies-a-visual-guide-with-code-8bc8fae71e1a
def plot_roc(ax, X_train, y_train, X_test, y_test, title):
    model = LogisticRegression(random_state=42)
    model.fit(X_train, y_train)
    y_test_pred = model.predict_proba(X_test)[:,1]
    fpr, tpr, thresh = metrics.roc_curve(y_test, y_test_pred)
    auc = metrics.roc_auc_score(y_test, y_test_pred)
    ax.plot(fpr,tpr,label=f"{title} AUC={auc:.3f}")
    ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')

    ax.set_title('ROC Curve')
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.legend(loc=0)

In [ ]:
plot_balance(df_train)

En el gráfico de arriba se puede ver como en el dataframe original la cantidad de muestras para los dias de lluvia son mucho menor que la de los días que no llovieron.

Comparamos con AUC que técnica de balanceo de datos es mejor para la regresión logística

In [ ]:
# Separación de variables explicativas y variables objetivo
X_train_balance = df_train.drop(['RainTomorrow'], axis=1).copy()
y_train_balance = df_train['RainTomorrow'].copy()

X_test_balance = df_test.drop(['RainTomorrow'], axis=1).copy()
y_test_balance = df_test['RainTomorrow'].copy()

### Subsampleo

In [ ]:
rus = RandomUnderSampler(random_state=0)
X_train_rus, y_train_rus = rus.fit_resample(X_train_balance, y_train_balance)

In [ ]:
df_rus = pd.DataFrame(X_train_rus, columns=X_train_balance.columns)
df_rus['RainTomorrow'] = y_train_rus

In [ ]:
plot_balance(df_rus)

Ahora en total tenemos 10004 muestras distribuidas igualitariamente entre los dias que llueve y los que no

In [ ]:
fig,ax1 = plt.subplots(1,1,figsize=(8,5))
plot_roc(ax1, X_train_rus, y_train_rus, X_test_balance, y_test_balance, 'Random Subsampleo')

#### - Balanceo con SMOTE

In [ ]:
smote = SMOTE(random_state=0)
X_train_smote, y_train_smote = smote.fit_resample(X_train_balance, y_train_balance)

In [ ]:
df_SMOTE = pd.DataFrame(X_train_smote, columns=X_train_balance.columns)
df_SMOTE['RainTomorrow'] = y_train_smote

In [ ]:
plot_balance(df_SMOTE)

Para esta técnica logramos tener 33176 muestras para cada clase

In [ ]:
fig,ax1 = plt.subplots(1,1,figsize=(8,5))
plot_roc(ax1, X_train_smote, y_train_smote, X_test_balance, y_test_balance, 'SMOTE Dataset')

#### - Balanceo con SMOTE(oversampling)-TOMEK(undersampling)

Además, investigamos otras maneras de balancear el DataSet y nos encontramos con SMOTE-TOMEK. Esta técnica combina el sobresampleo con SMOTE con el subsampleo con TOMEK Link que toma dos puntos cercanos de diferentes clases y por cada par se borran aquellos que pertenezcan a la clase mayoritoria, de esa forma el límite de decisión se vuelve mas claro.

  Fuente: https://towardsdatascience.com/class-imbalance-strategies-a-visual-guide-with-code-8bc8fae71e1a

In [ ]:
smotetomek = SMOTETomek(random_state=0)
X_train_smotetomek, y_train_smotetomek = smotetomek.fit_resample(X_train_balance, y_train_balance)

In [ ]:
df_smotetomek = pd.DataFrame(X_train_smotetomek, columns=X_train_balance.columns)
df_smotetomek['RainTomorrow'] = y_train_smotetomek

In [ ]:
plot_balance(df_smotetomek)

Se logran tener 33056 muestras en total.

In [ ]:
fig,ax1 = plt.subplots(1,1,figsize=(8,5))
plot_roc(ax1, X_train_smotetomek, y_train_smotetomek, X_test_balance, y_test_balance, 'SMOTE-TOMEK Dataset')

In [ ]:
# VARIABILIDAD DE LOS DATASETS BALANCEADOS CON: Oversampling, SMOTE, SMOTE-Tomek
std_combined = pd.DataFrame([df_rus.describe().loc['std'], df_SMOTE.describe().loc['std'], df_smotetomek.describe().loc['std'], df_test.describe().loc['std']]).T
std_combined.columns = ['Undersampling', 'SMOTE', 'SMOTE-Tomek', 'Original']
std_combined

Los tres datasets tiene una variabilidad muy similar, lo que explica porqué la performace es muy parecida. Optamos por usar el Dataset balanceado con la técnica SMOTE

# Métricas regresion logistica con datos balanceado

In [ ]:
def plot_matrix_confusion(y_test, y_pred):
  # Calcular la matriz de confusión
  confusion_mat = confusion_matrix(y_test, y_pred)

  # Crear una figura y un eje para la matriz de confusión
  plt.figure(figsize=(4, 3))
  sns.heatmap(confusion_mat, annot=True, fmt="d", cmap="Blues", linewidths=0.1, linecolor='blue', cbar=True)

  # Añadir etiquetas a los ejes y al gráfico
  plt.xlabel('Predicción')
  plt.ylabel('Verdaderos')
  plt.title('Matriz de Confusión')
  plt.xticks(np.arange(2) + 0.5, ['No', 'Sí'])
  plt.yticks(np.arange(2) + 0.5, ['No', 'Sí'])
  plt.show()

In [ ]:
def train_test_with_balanced_df(df_train, df_test, model, title=None):
  """ Entrena, muestra métricas y plotea la curva AUC """
  X_train = df_train.drop(['RainTomorrow'], axis=1).copy()
  y_train = df_train['RainTomorrow'].copy()

  X_test = df_test.drop(['RainTomorrow'], axis=1).copy()
  y_test = df_test['RainTomorrow'].copy()

  my_model = model(random_state=42)
  my_model.fit(X_train, y_train)
  y_pred = my_model.predict(X_test)


  print(f"Resultados con: {model.__name__}\n")
  print(classification_report(y_test, y_pred))
  print(confusion_matrix(y_test, y_pred))

  print(f"\n\nMatriz de confusión para: {model.__name__}")
  plot_matrix_confusion(y_test, y_pred)

  fig,ax = plt.subplots(1,1,figsize=(8,5))
  y_test_pred = my_model.predict_proba(X_test)[:,1]
  fpr, tpr, thresh = metrics.roc_curve(y_test, y_test_pred)

  # Posicion del indice del umbral 0.5
  thresh_index = np.abs(thresh - 0.5).argmin()
  # Posicion del indice del umbral con valores mas cercanos a (0,1) para TPR y FPR
  optimal_index = ((1 - tpr**2) + fpr**2).argmin()


  auc = metrics.roc_auc_score(y_test, y_test_pred)
  ax.plot(fpr,tpr,label=f"Logistic Regression AUC={auc:.3f}")
  ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')

  ax.set_title(f'{title}')
  ax.set_xlabel('False Positive Rate')
  ax.set_ylabel('True Positive Rate')
  ax.legend(loc=0)

  print(f"\n\nUmbrales:")
  print(f"Actual: {thresh[thresh_index]}")
  print(f"Optimo: {thresh[optimal_index]}")

  print(f"\n\nPuntaje para training vs test")
  print(f"Training: {my_model.score(X_train, y_train)}")
  print(f"Test: {my_model.score(X_test, y_test)}")

In [ ]:
train_test_with_balanced_df(df_rus, df_test, LogisticRegression, "AUC Curva: TODAS LAS VARIABLES")

Un umbral de 0.5 no es óptimo para este problema, se logran mejores métricas con uno de 0.3.
Logramos un buen fitteo de los datos porque nuestro modelo performa mejor para los datos de test(0.796) que los de train (0.791)

# Modelo base Regresion Lineal

In [ ]:
# Separación de variables explicativas y variables objetivo
X = df.drop(['RainfallTomorrow'], axis=1).copy()
y = df[['RainfallTomorrow']].copy()

# Spliteo mi dataset en train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape


In [ ]:
# Creo un Dataframe de TRAIN
df_train = pd.DataFrame(X_train, columns=X.columns)
df_train['RainfallTomorrow'] = y['RainfallTomorrow']

In [ ]:
# Creo un Dataframe de TEST
df_test = pd.DataFrame(X_test, columns=X.columns)
df_test['RainfallTomorrow'] = y['RainfallTomorrow']

In [ ]:
# Preproceso mi df de test y mi df de train
df_train = preprocessor.fit_transform(df_train)
df_test = preprocessor.fit_transform(df_test)

In [ ]:
X_base_regresion_train = df_train[['RainToday', 'Sunshine']]
Y_base_regresion_train = df_train["RainfallTomorrow"]

In [ ]:
X_base_regresion_test = df_test[['RainToday', 'Sunshine',]]
Y_base_regresion_test = df_test["RainfallTomorrow"]

In [ ]:
modelo_base_regresion = LinearRegression()

In [ ]:
modelo_base_regresion.fit(X_base_regresion_train, Y_base_regresion_train)

In [ ]:
y_pred_base_regresion = modelo_base_regresion.predict(X_base_regresion_test)

In [ ]:
def errores(y_test, y_pred):
    print("R2:   %.2f" % r2_score(y_test, y_pred))
    print("MSE:  %.2f" % mean_squared_error(y_test, y_pred))
    print("RMSE: %.2f" % sqrt(mean_squared_error(y_test, y_pred)))
    print("MAE:  %.2f" % mean_absolute_error(y_test, y_pred))
    print("MAPE: %.2f" % mean_absolute_percentage_error(y_test, y_pred))


errores(Y_base_regresion_test, y_pred_base_regresion)

EL modelo base (seleccionando solamente dos variables) para nuestro problema de regresión lineal tiene métricas no muy buenas, un valor MSE alejado de cero lo que indica que no es muy exacto. Un r2 muy bajo por lo que tambien indica que no explica tampoco bien la variabilidad de nuestro dataset

# Base Regresión - Importado de scikit learn

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html#sklearn-dummy-dummyclassifier
from sklearn.dummy import DummyRegressor

df_base=preprocessor.fit_transform(df)
X=df_base.drop(['RainTomorrow'], axis=1)
y=df_base['RainTomorrow']
y_reg=df_base['RainfallTomorrow']
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, stratify=y, random_state=777)
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y_reg,test_size=0.3, stratify=y, random_state=777)

In [ ]:
dummy_regr = DummyRegressor(strategy="mean")

dummy_regr.fit(X_train_reg, y_train_reg)

y_pred = dummy_regr.predict(X_test_reg)

# Metricas
mse = mean_squared_error(y_test_reg, y_pred)

r2 = r2_score(y_test_reg, y_pred)

print(f"Mean squared error: {mse}")
print(f"R-squared score: {r2}")

In [ ]:
# Plot the predicted values against the actual values
plt.scatter(y_test_reg, y_pred)
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Dummy Regressor: Predicted vs. Actual")
# Add a diagonal line to show where the predicted values match the actual values
plt.plot([min(y_test_reg), max(y_test_reg)], [min(y_test_reg), max(y_test_reg)], color="red")

Tampoco es un buen modelo base predecir con la media de los datos, el mse para este modelo es mayor a uno y un r2 negativo indica que no explica en absoluto la variabilidad de nuestros datos

# Modelo Base clasificación

In [ ]:
# Separación de variables explicativas y variables objetivo
X = df.drop(['RainTomorrow', 'RainfallTomorrow'], axis=1).copy()
y = df[['RainTomorrow']].copy()

# Spliteo mi dataset en train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# Creo un Dataframe de TRAIN
df_train = pd.DataFrame(X_train, columns=X.columns)
df_train['RainTomorrow'] = y['RainTomorrow']

In [ ]:
# Creo un Dataframe de TEST
df_test = pd.DataFrame(X_test, columns=X.columns)
df_test['RainTomorrow'] = y['RainTomorrow']

In [ ]:
# Preproceso mi df de test y mi df de train
df_train = preprocessor.fit_transform(df_train)
df_test = preprocessor.fit_transform(df_test)

In [ ]:
X_base_clasificacion_train = df_train[['MaxTemp', 'Sunshine', 'MinTemp']]
Y_base_clasificacion_train = df_train["RainTomorrow"]

In [ ]:
X_base_clasificacion_test = df_test[['MaxTemp', 'Sunshine', 'MinTemp']]
Y_base_clasificacion_test = df_test["RainTomorrow"]

In [ ]:
modelo_base_clasificacion = LogisticRegression()

In [ ]:
modelo_base_clasificacion.fit(X_base_clasificacion_train, Y_base_clasificacion_train)

In [ ]:
y_pred_base_clasificacion = modelo_base_clasificacion.predict(X_base_clasificacion_test)

In [ ]:
print("Logistica Modelo base\n")
print(classification_report(Y_base_clasificacion_test, y_pred_base_clasificacion))
ConfusionMatrixDisplay(confusion_matrix(Y_base_clasificacion_test, y_pred_base_clasificacion)).plot()



Nuestro modelo base de regresión, tiene una buena performance ya que al rededor del 78% de los datos son categorizados correctamente

# Base Clasificación - Importado de scikit learn

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html#sklearn-dummy-dummyclassifier
from sklearn.dummy import DummyClassifier


df_base=preprocessor.fit_transform(df)
X=df_base.drop(['RainTomorrow'], axis=1)
y=df_base['RainTomorrow']
y_reg=df_base['RainfallTomorrow']
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, stratify=y, random_state=777)
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y_reg,test_size=0.3, stratify=y, random_state=777)

In [ ]:
# Crear un clasificador de la mayoría
dummy_clf = DummyClassifier(strategy="most_frequent")

# Entreno, aunque no es necesario porque no aprende nada
dummy_clf.fit(X_train, y_train)

# Hacer predicciones
y_pred = dummy_clf.predict(X_test)

# Score
dummy_clf.score(X_test, y_test)

# Metricas
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

La presición obtenida es equivalente a predecir siempre que no lloverá, habría asertado un 76% de los casos según nuestro data set.

El cual podemos verificarlo rápidamente contando la cantidad de cada valor de la variable a predecir.  Nuestro modelo dirá todas las predicciones que NO lloverá pero éste tendra una eficiencia del 76%.


In [ ]:
df_base.value_counts("RainTomorrow", normalize=True)

# Optimizacion de parametros

In [ ]:
from sklearn.model_selection import GridSearchCV

### Analisis de los HP - GridCV

In [ ]:
### Para cada hiperparámetro se chequean las métricas de Precisión y Recall.
scores = ["precision", "recall"]

In [ ]:
def print_dataframe(filtered_cv_results):
    ### Un print bonito de los resultados de cross validation
    for mean_precision, std_precision, mean_recall, std_recall, params in zip(
        filtered_cv_results["mean_test_precision"],
        filtered_cv_results["std_test_precision"],
        filtered_cv_results["mean_test_recall"],
        filtered_cv_results["std_test_recall"],
        filtered_cv_results["params"],
    ):
        print(
            f"precision: {mean_precision:0.3f} (±{std_precision:0.03f}),"
            f" recall: {mean_recall:0.3f} (±{std_recall:0.03f}),"
            f" for {params}"
        )
    print()

In [ ]:
def refit_strategy(cv_results):
    """Define the strategy to select the best estimator.

    The strategy defined here is to filter-out all results below a precision threshold
    of 0.7, rank the remaining by recall and keep all models with one standard
    deviation of the best by recall. Once these models are selected, we can select the
    fastest model to predict.

    Parameters
    ----------
    cv_results : dict of numpy (masked) ndarrays
        CV results as returned by the `GridSearchCV`.

    Returns
    -------
    best_index : int
        The index of the best estimator as it appears in `cv_results`.
    """
    # print the info about the grid-search for the different scores
    precision_threshold = 0.7

    cv_results_ = pd.DataFrame(cv_results)
    print("All grid-search results:")
    print_dataframe(cv_results_)

    # Filter-out all results below the threshold
    high_precision_cv_results = cv_results_[
        cv_results_["mean_test_precision"] > precision_threshold
    ]

    print(f"Models with a precision higher than {precision_threshold}:")
    print_dataframe(high_precision_cv_results)

    high_precision_cv_results = high_precision_cv_results[
        [
            "mean_score_time",
            "mean_test_recall",
            "std_test_recall",
            "mean_test_precision",
            "std_test_precision",
            "rank_test_recall",
            "rank_test_precision",
            "params",
        ]
    ]

    # Select the most performant models in terms of recall
    # (within 1 sigma from the best)
    best_recall_std = high_precision_cv_results["mean_test_recall"].std()
    best_recall = high_precision_cv_results["mean_test_recall"].max()
    best_recall_threshold = best_recall - best_recall_std

    high_recall_cv_results = high_precision_cv_results[
        high_precision_cv_results["mean_test_recall"] > best_recall_threshold
    ]
    print(
        "Out of the previously selected high precision models, we keep all the\n"
        "the models within one standard deviation of the highest recall model:"
    )
    print_dataframe(high_recall_cv_results)

    # From the best candidates, select the fastest model to predict
    fastest_top_recall_high_precision_index = high_recall_cv_results[
        "mean_score_time"
    ].idxmin()

    print(
        "\nThe selected final model is the fastest to predict out of the previously\n"
        "selected subset of best models based on precision and recall.\n"
        "Its scoring time is:\n\n"
        f"{high_recall_cv_results.loc[fastest_top_recall_high_precision_index]}"
    )

    return fastest_top_recall_high_precision_index

In [ ]:
# Spliteo mi dataset en train-test
X_train_gs = df_test.drop(['RainTomorrow'], axis=1)
y_train_gs = df_test['RainTomorrow']

In [ ]:
tuned_parameters = [
    {"class_weight": [None, 'balanced'], "C": [0.1, 1, 10,100], "solver": ['newton-cg'], "max_iter": [200, 500]}
]

grid_search = GridSearchCV(
    LogisticRegression(), tuned_parameters, scoring=scores, refit=refit_strategy
)
grid_search.fit(X_train_gs, y_train_gs)

In [ ]:
grid_search.best_params_

In [ ]:
y_pred = grid_search.predict(df_test.drop(['RainTomorrow'], axis=1))
print(classification_report(df_test['RainTomorrow'], y_pred))

Con la optimización de hiperparámetros usando gridcv, nos muestra que los mejores hiperparametros para nuestro modelo de regresión logística con buen score de precision y recall son los siguentes:
<br>
No hace falta que el data set esté balanceado, converge en 200 iteraciones y con un valor de regularización bajo|.

Por default en LogisticRegression de scikit-learn por default usa un valor
C= 1.0
ClassWeight=None, que no es optimizado
max_iter=100
y seteamos el solver para que solo pruebe con newton-cg pero por default usa 'lbfgs'

# Validación Cruzada

Consideramos necesario usar validación cruzada para evaluar mejor la performance del modelo con los mejores parametros

In [ ]:
classification = LogisticRegression(C=10, class_weight='balanced', max_iter=500, solver='newton-cg')

In [ ]:
df_test_cv = pd.concat([df_test, df_train], ignore_index=True)

X_cv, y_cv = df_test_cv.drop(['RainTomorrow'], axis=1), df_test_cv['RainTomorrow']

In [ ]:
scores = cross_val_score(classification, X_cv, y_cv, cv=KFold(n_splits=5, shuffle=True, random_state=42))
scores

In [ ]:
cv_strategies = [
    ("KFold", KFold(n_splits=5, shuffle=True, random_state=42))
]

results = {}
for name, cv in cv_strategies:
    scores = cross_val_score(classification, X_cv, y_cv, cv=cv)
    results[name] = scores

In [ ]:
plt.figure(figsize=(10, 6))
for name, scores in results.items():
    plt.plot(range(1, len(scores) + 1), scores, marker='o', label=name)

plt.xlabel("Fold")
plt.ylabel("Accuracy")
plt.title("Estrategias de validación cruzada")
plt.legend()
plt.grid(True)
plt.show()

En k-folds, los scores no varían demasiado por lo que indica que nuestro modelo tiene robustez

# Redes Neuronales

## Redes Neuronales -- Regresión

In [ ]:
# Dropeo valores nulos de 'RainfallTomorrow' de mi dataframe original
df.dropna(subset=['RainToday', 'RainTomorrow'], inplace=True)

# Separación de variables explicativas y variables objetivo
X = df.drop(['RainfallTomorrow'], axis=1).copy()
y = df[['RainfallTomorrow']].copy()

# Spliteo mi dataset en train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# DF train
df_train = pd.DataFrame(X_train, columns=X.columns)
df_train['RainfallTomorrow'] = y['RainfallTomorrow']

# DF test
df_test = pd.DataFrame(X_test, columns=X.columns)
df_test['RainfallTomorrow'] = y['RainfallTomorrow']

# Preproceso mi df de test y mi df de train
df_train = preprocessor.fit_transform(df_train)
df_test = preprocessor.fit_transform(df_test)

In [ ]:
X_train_regresion = df_train.drop(['RainfallTomorrow', 'RainTomorrow'], axis=1)
y_train_regresion = df_train['RainfallTomorrow']

X_test_regresion = df_test.drop(['RainfallTomorrow', 'RainTomorrow'], axis=1)
y_test_regresion = df_test['RainfallTomorrow']

In [ ]:
def objective_reg(trial: Trial):

    # Creamos el modelo
    model = Sequential()

    # Definimos HP a optimizar
    num_layers = trial.suggest_int('num_layers', 1, 5)
    num_units_input = trial.suggest_int('n_units_input', 4, 128)
    model.add(Dense(num_units_input, activation='relu', input_shape=(X_train_regresion.shape[1],)))

    for i in range(num_layers):
        num_units = trial.suggest_int(f'n_units_layer_{i}', 4, 128)
        model.add(Dense(num_units, activation='relu'))
        dropout_rate = trial.suggest_float(f'dropout_rate_layer_{i}', 0.0, 0.5)
        model.add(Dropout(dropout_rate))

    model.add(Dense(1, activation='linear'))  # Capa final para la regresión

    # Compilamos el modelo
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')

    # Split train, val test
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_regresion, y_train_regresion, test_size=0.2, random_state=42)

    # Entrenamos el modelo
    model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=10, batch_size=64, verbose=0)

    # Evaluamos el modelo
    score = model.evaluate(X_valid, y_valid, verbose=0)
    return score

In [ ]:
study = optuna.create_study(direction='minimize')  # Minimizar la pérdida
study.optimize(objective_reg, n_trials=10)

In [ ]:
best_params = study.best_params
print("Best parameters:", best_params)

In [ ]:
# Entrenamos el modelo
best_model = Sequential()
best_model.add(Dense(best_params['n_units_input'], activation='relu', input_shape=(X_train_regresion.shape[1],)))


for i in range(best_params['num_layers']):
    best_model.add(Dense(best_params[f'n_units_layer_{i}'], activation='relu'))
    best_model.add(Dropout(best_params[f'dropout_rate_layer_{i}']))

best_model.add(Dense(1, activation='linear'))

In [ ]:
best_model.compile(optimizer=Adam(learning_rate=best_params['learning_rate']), loss='mean_squared_error')

In [ ]:
# Entrenamos el MEJOR modelo
best_model.fit(X_train_regresion, y_train_regresion, epochs=10, batch_size=64, verbose=0)

In [ ]:
# Evaluamos el MEJOR modelo
train_scores = best_model.evaluate(X_train_regresion, y_train_regresion, verbose=0)
valid_scores = best_model.evaluate(X_test_regresion, y_test_regresion, verbose=0)

train_r2 = r2_score(y_train_regresion, best_model.predict(X_train_regresion))
valid_r2 = r2_score(y_test_regresion, best_model.predict(X_test_regresion))

train_mse = mean_squared_error(y_train_regresion, best_model.predict(X_train_regresion))
valid_mse = mean_squared_error(y_test_regresion, best_model.predict(X_test_regresion))

In [ ]:
# Chequeamos sus métricas
print(f"Train R2: {train_r2}, Test R2: {valid_r2}")
print(f"Train MSE: {train_mse}, Test MSE: {valid_mse}")

El valor de r2 no es muy cercano a uno, en general explica alrededor del 35% de la variabilidad en los datos de entrenamiento y prueba.


In [ ]:
# Predicciones
y_pred = best_model.predict(X_test_regresion)

# Metricas
print(f"R2: {r2_score(y_test_regresion, y_pred)}")
print(f"MSE: {mean_squared_error(y_test_regresion, y_pred)}")

En comparación con el mejor modelo de regresión linear que fue la RN con gradiente descendiente, la red neuronal ajusta mejor a los datos

# Explicabilidad - NN Regresión

In [ ]:
X_train_regresion = X_train_regresion.sample(frac=.25, random_state=123)
X_test_regresion = X_test_regresion.sample(frac=.25, random_state=123)

In [ ]:
explainer = shap.Explainer(best_model,
                            X_train_regresion.values,
                            feature_names=X_train_regresion.columns)

explanation = explainer(X_test_regresion)

In [ ]:
# Prevenir error de dimensiones, porque explica ambas clases: https://stackoverflow.com/questions/73605032/error-in-shap-plots-tree-explainer-when-using-sklearn-pipeline-for-classificat
shap.plots.beeswarm(explanation, max_display=None)

Para las redes neuronales, las variables mas explicativas son La humedad de las 3pm, La presion de las 3pm y la presion de las 9am. Tanto la presión como la humedad, a mayor valor mayor es la cantidad de lluvia que predice mi modelo. En cambio para la presion de las 3pm a mayor valor, en el modelo resta la cantidad de lluvia. Esto lo concluimos con los resultado de la primera vez que creamos la red.

## Explicabilidad local NN Regresión

In [ ]:
x_train_array = X_train_regresion.sample(150)
x_test_array = X_test_regresion.sample(150)

In [ ]:
explainer = shap.KernelExplainer(best_model, shap.sample(x_train_array,90))

In [ ]:
muestra = x_test_array.iloc[0:1].values
shap_values = explainer.shap_values(muestra)

base_value = explainer.expected_value

pr_proba = best_model.predict(x_test_array)[0]
result = np.argmax(pr_proba)

In [ ]:
explanation = shap.Explanation(shap_values, base_value)
shap.summary_plot(explanation, x_test_array, feature_names=X_train_regresion.columns)
shap.plots.bar(explanation)

Conclusión: Al comparar los valores de MSE, para el problema de regresión, el mejor modelo es el de Redes Neuronales con los hiperparámetros optimizados. Anteriormente el mejor modelo había sido el de Regresión Lineal con gradiente descendiente con un valor de MSE igual a 0.76 mientra que el de las NN es igual a 0.67

## Redes Neuronales -- Clasificación

In [ ]:
# Dropeo valores nulos de 'RainfallTomorrow' de mi dataframe original
df.dropna(subset=['RainToday', 'RainTomorrow'], inplace=True)

# Separación de variables explicativas y variables objetivo
X = df.drop(['RainfallTomorrow', 'RainTomorrow'], axis=1).copy()
y = df[['RainTomorrow']].copy()

# Spliteo mi dataset en train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# DF train
df_train = pd.DataFrame(X_train, columns=X.columns)
df_train['RainTomorrow'] = y['RainTomorrow']

# DF test
df_test = pd.DataFrame(X_test, columns=X.columns)
df_test['RainTomorrow'] = y['RainTomorrow']

# Preproceso mi df de test y mi df de train
df_train = preprocessor.fit_transform(df_train)
df_test = preprocessor.fit_transform(df_test)

In [ ]:
X_train_clasificacion  = df_train.drop('RainTomorrow', axis=1)
y_train_clasificacion  = df_train['RainTomorrow']

X_test_clasificacion  = df_test.drop('RainTomorrow', axis=1)
y_test_clasificacion  = df_test['RainTomorrow']

In [ ]:
def objective_class(trial: Trial):
    # Create the model
    model = Sequential()
    num_layers = trial.suggest_int('num_layers', 1, 5)

    for i in range(num_layers):
        num_units = trial.suggest_int(f'n_units_layer_{i}', 4, 128)
        model.add(Dense(num_units, activation='relu'))
        dropout_rate = trial.suggest_float(f'dropout_rate_layer_{i}', 0.0, 0.5)
        model.add(Dropout(dropout_rate))


    model.add(Dense(2, activation='sigmoid'))  # Output layer for multi-class classification

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[Precision()])

    # Split test, val, train
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_clasificacion, y_train_clasificacion, test_size=0.2, random_state=42)


    # Convertir etiquetas a one-hot encoding
    label_encoder = LabelEncoder()
    y_train = to_categorical(label_encoder.fit_transform(y_train))
    y_valid = to_categorical(label_encoder.transform(y_valid))

    # Entreno
    model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=5, batch_size=64, verbose=0)

    # Fitteo
    score = model.evaluate(X_valid, y_valid, verbose=0)
    return score[1]  # Devuelve precision

In [ ]:
# Otimizacion de HP
study = optuna.create_study(direction='maximize')  # Minimizar la pérdida
study.optimize(objective_class, n_trials=10)

In [ ]:
# Obtenemos los mejores parametros
best_params = study.best_params
print("Best parameters:", best_params)

In [ ]:
best_model = Sequential()
best_model.add(Dense(2, activation='relu', input_shape=(X_train_clasificacion.shape[1],)))

for i in range(best_params['num_layers']):
    best_model.add(Dense(best_params[f'n_units_layer_{i}'], activation='relu'))
    best_model.add(Dropout(best_params[f'dropout_rate_layer_{i}']))

best_model.add(Dense(2, activation='softmax'))

In [ ]:
best_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[Precision()])

In [ ]:
label_encoder = LabelEncoder()
y_train_clasificacion = to_categorical(label_encoder.fit_transform(y_train_clasificacion))
y_test_clasificacion = to_categorical(label_encoder.transform(y_test_clasificacion))

In [ ]:
# Entrenamos
best_model.fit(X_train_clasificacion, y_train_clasificacion, epochs=20, batch_size=64, verbose=0)

In [ ]:
# Evaluamos
train_scores = best_model.evaluate(X_train_clasificacion, y_train_clasificacion, verbose=0)
valid_scores = best_model.evaluate(X_test_clasificacion, y_test_clasificacion, verbose=0)

train_preds = np.argmax(best_model.predict(X_train_clasificacion), axis=1)
valid_preds = np.argmax(best_model.predict(X_test_clasificacion), axis=1)

train_labels = np.argmax(y_train_clasificacion, axis=1)
valid_labels = np.argmax(y_test_clasificacion, axis=1)

train_accuracy = accuracy_score(train_labels, train_preds)
valid_accuracy = accuracy_score(valid_labels, valid_preds)

In [ ]:
# Metricas
print(f"Train Accuracy: {train_accuracy}, Validation Accuracy: {valid_accuracy}")

In [ ]:
if len(y_test_clasificacion.shape) > 1 and y_test_clasificacion.shape[1] > 1:
    y_test_clasificacion = np.argmax(y_test_clasificacion, axis=1)

In [ ]:
predictions_prob = best_model.predict(X_test_clasificacion)

predictions = (predictions_prob > 0.5).astype('int32')  # Convertir probabilidades a etiquetas binarias
if len(predictions.shape) > 1 and predictions.shape[1] > 1:
    predictions = np.argmax(predictions, axis=1)

conf_matrix = confusion_matrix(y_test_clasificacion, predictions)

In [ ]:
ConfusionMatrixDisplay(conf_matrix).plot()

In [ ]:
# Print recall, accuracy of the neuralnetwork
from sklearn.metrics import recall_score, accuracy_score, precision_score

print(f"La red Neuronal predice:\n -Correctamente la días de lluvia y no lluvia en un {accuracy_score(y_test_clasificacion, predictions)} (accuracy)\n -De todos los días de lluvia, le acierta al: {recall_score(y_test_clasificacion, predictions)} (recall)\n -De todos los días de no lluvia, le acierta al {precision_score(y_test_clasificacion, predictions)} (precision) ")

In [ ]:
print(type(best_model))

# Explicabilidad RN Clasificación

Se necesita la version de python 3.10 para correrlo

In [ ]:
x_train_clasi_array = X_train_clasificacion.sample(frac=0.003, random_state=123)
x_test_clasi_array = X_test_clasificacion.sample(frac=0.003, random_state=123)

In [ ]:
explainer = shap.KernelExplainer(best_model,
                            x_train_clasi_array.values,
                            feature_names=x_train_clasi_array.columns)

In [ ]:
shap_values = explainer.shap_values(x_train_clasi_array)

In [ ]:
explanation = explainer(x_train_clasi_array)

In [ ]:
shap.summary_plot(shap_values, x_train_clasi_array.values, feature_names = x_train_clasi_array.columns, class_names=['Llueve', 'No llueve'])

Conclusión: Al comparar los valores de Accuracy, para el problema de clasificación, el mejor modelo es el de Regresión Logística con los hiperparametros optimizados con una Accuracy del 85% seguido por las Redes Neuronales con los hiperparámetros optimizados que tiene un 84%